#Jukebox notebook with autosaving.

Speed upsampling supported. Switch to upsample mode will happen automatically if data file is detected within the folder provided.

Colab Pro users can use the 5b_lyrics (or 5b) model.
Free users can also use the 5b_lyrics (or 5b) model, if they are assigned a Tesla T4 GPU. If assigned a Tesla K80, the weaker 1b_lyrics model is recommended.

Join the Jukebox community at https://discord.gg/aEqXFN9amV

This Notebook Was edited by Yo Jeremijenko-Conley to generate music for a project for MUSIGR6610 - SOUND: ADVANCED TOPICS at Columbia University

In [ ]:
#@title Check which GPU you were assigned by running this cell.
!nvidia-smi -L
your_lyrics = """
"""

GPU 0: Tesla T4 (UUID: GPU-62e599f3-654a-1ad9-f5c7-1737ca6595a6)


In [ ]:
your_lyrics = """
"""

In [ ]:
#@title Select your settings and run this cell to start generating

from google.colab import drive
drive.mount('/content/gdrive')

!pip install --upgrade git+https://github.com/craftmine1000/jukebox-saveopt.git

import jukebox
import torch as t
import librosa
import os
from IPython.display import Audio
from jukebox.make_models import make_vqvae, make_prior, MODELS, make_model
from jukebox.hparams import Hyperparams, setup_hparams
from jukebox.sample import sample_single_window, _sample, \
                           sample_partial_window, upsample, \
                           load_prompts
from jukebox.utils.dist_utils import setup_dist_from_mpi
from jukebox.utils.torch_utils import empty_cache
# MPI Connect. MPI doesn't like being initialized twice, hence the following
try:
    if device is not None:
        pass
except NameError:
    rank, local_rank, device = setup_dist_from_mpi()

model = "5b" #@param ["5b_lyrics", "5b", "1b_lyrics"]
if model == '5b':
  your_lyrics = """
  """

save_and_load_models_from_drive = False

#START GDRIVE MODEL LOADER
if save_and_load_models_from_drive == True:
  import os.path
  !apt install pv
  !mkdir /root/.cache ; mkdir /root/.cache/jukebox ; mkdir /root/.cache/jukebox/models ; mkdir /root/.cache/jukebox/models/1b_lyrics ; mkdir /root/.cache/jukebox/models/5b_lyrics ; mkdir /root/.cache/jukebox/models/5b
  !mkdir /content/gdrive/MyDrive/jukebox
  !mkdir /content/gdrive/MyDrive/jukebox/models
  !mkdir /content/gdrive/MyDrive/jukebox/models/5b
  !mkdir /content/gdrive/MyDrive/jukebox/models/5b_lyrics
  !mkdir /content/gdrive/MyDrive/jukebox/models/1b_lyrics


def load_5b_vqvae():
    if os.path.exists("/root/.cache/jukebox/models/5b/vqvae.pth.tar") == False:
      if os.path.exists("/content/gdrive/MyDrive/jukebox/models/5b/vqvae.pth.tar") == False:
        print("5b_vqvae not stored in Google Drive. Downloading for the first time.")
        !wget https://openaipublic.azureedge.net/jukebox/models/5b/vqvae.pth.tar -O /content/gdrive/MyDrive/jukebox/models/5b/vqvae.pth.tar
      else:
        print("5b_vqvae stored in Google Drive.")
      print('Copying 5b VQVAE')
      !pv /content/gdrive/MyDrive/jukebox/models/5b/vqvae.pth.tar > /root/.cache/jukebox/models/5b/vqvae.pth.tar

def load_1b_lyrics_level2():
  if os.path.exists("/root/.cache/jukebox/models/1b_lyrics/prior_level_2.pth.tar") == False:
    if os.path.exists("/content/gdrive/MyDrive/jukebox/models/1b_lyrics/prior_level_2.pth.tar") == False:
      print("1b_lyrics_level_2 not stored in Google Drive. Downloading for the first time. This will take a few more minutes.")
      !wget https://openaipublic.azureedge.net/jukebox/models/1b_lyrics/prior_level_2.pth.tar -O /content/gdrive/MyDrive/jukebox/models/1b_lyrics/prior_level_2.pth.tar
    else:
      print("1b_lyrics_level_2 stored in Google Drive.")
    print("Copying 1B_Lyrics Level 2")
    !pv /content/gdrive/MyDrive/jukebox/models/1b_lyrics/prior_level_2.pth.tar > /root/.cache/jukebox/models/1b_lyrics/prior_level_2.pth.tar

def load_5b_lyrics_level2():
  if os.path.exists("/root/.cache/jukebox/models/5b_lyrics/prior_level_2.pth.tar") == False:
    if os.path.exists("/content/gdrive/MyDrive/jukebox/models/5b_lyrics/prior_level_2.pth.tar") == False:
      print("5b_lyrics_level_2 not stored in Google Drive. Downloading for the first time. This will take up to 10-15 minutes.")
      !wget https://openaipublic.azureedge.net/jukebox/models/5b_lyrics/prior_level_2.pth.tar -O /content/gdrive/MyDrive/jukebox/models/5b_lyrics/prior_level_2.pth.tar
    else:
      print("5b_lyrics_level_2 stored in Google Drive.")
    print("Copying 5B_Lyrics Level 2")
    !pv /content/gdrive/MyDrive/jukebox/models/5b_lyrics/prior_level_2.pth.tar > /root/.cache/jukebox/models/5b_lyrics/prior_level_2.pth.tar

def load_5b_level1():
  if os.path.exists('/root/.cache/jukebox/models/5b/prior_level_1.pth.tar') == False:
    if os.path.exists("/content/gdrive/MyDrive/jukebox/models/5b/prior_level_1.pth.tar") == False:
      print("5b_level_1 not stored in Google Drive. Downloading for the first time. This may take a few more minutes.")
      !wget https://openaipublic.azureedge.net/jukebox/models/5b/prior_level_1.pth.tar -O /content/gdrive/MyDrive/jukebox/models/5b/prior_level_1.pth.tar
    else:
      print("5b_level_1 stored in Google Drive.")
    print("Copying 5B Level 1")
    !pv /content/gdrive/MyDrive/jukebox/models/5b/prior_level_1.pth.tar > /root/.cache/jukebox/models/5b/prior_level_1.pth.tar

def load_5b_level0():
  if os.path.exists('/root/.cache/jukebox/models/5b/prior_level_0.pth.tar') == False:
    if os.path.exists("/content/gdrive/MyDrive/jukebox/models/5b/prior_level_0.pth.tar") == False:
      print("5b_level_0 not stored in Google Drive. Downloading for the first time. This may take a few minutes.")
      !wget https://openaipublic.azureedge.net/jukebox/models/5b/prior_level_0.pth.tar -O /content/gdrive/MyDrive/jukebox/models/5b/prior_level_0.pth.tar
    else:
      print("5b_level_0 stored in Google Drive.")
    print("Copying 5B Level 0")
    !pv /content/gdrive/MyDrive/jukebox/models/5b/prior_level_0.pth.tar > /root/.cache/jukebox/models/5b/prior_level_0.pth.tar

def load_5b_level2():
  if os.path.exists('/root/.cache/jukebox/models/5b/prior_level_2.pth.tar') == False:
    if os.path.exists("/content/gdrive/MyDrive/jukebox/models/5b/prior_level_2.pth.tar") == False:
      print("5b_level_2 not stored in Google Drive. Downloading for the first time. This will take up to 10-15 minutes.")
      !wget https://openaipublic.azureedge.net/jukebox/models/5b/prior_level_2.pth.tar -O /content/gdrive/MyDrive/jukebox/models/5b/prior_level_2.pth.tar
    else:
      print("5b_level_2 stored in Google Drive.")
  print("Copying 5B Level 2")
  !pv /content/gdrive/MyDrive/jukebox/models/5b/prior_level_2.pth.tar > /root/.cache/jukebox/models/5b/prior_level_2.pth.tar

if save_and_load_models_from_drive == True:
  if model == '5b_lyrics':
    load_5b_vqvae()
    load_5b_lyrics_level2()
    load_5b_level1()
    load_5b_level0()
  if model == '5b':
    load_5b_vqvae()
    load_5b_level2()
    load_5b_level1()
    load_5b_level0()
  elif model == '1b_lyrics':
    load_5b_vqvae()
    load_1b_lyrics_level2()
    load_5b_level1()
    load_5b_level0()
#END GDRIVE MODEL LOADER

hps = Hyperparams()
hps.sr = 44100
hps.n_samples =  2#@param {type:"integer"}
# Specifies the directory to save the sample in.
# We set this to the Google Drive mount point.
hps.name = '/content/gdrive/MyDrive/musicAI/musicAI20' #@param {type:"string"}
chunk_size = 16 if model in ('5b', '5b_lyrics') else 32
gpu_info = !nvidia-smi -L
if gpu_info[0].find('Tesla T4') >= 0:
  max_batch_size = 2
  print('Tesla T4 detected, max_batch_size set to 2')
elif gpu_info[0].find('Tesla K80') >= 0:
  max_batch_size = 8
  print('Tesla K80 detected, max_batch_size set to 8')
elif gpu_info[0].find('Tesla P100') >= 0:
  max_batch_size = 3
  print('Tesla P100 detected, max_batch_size set to 3')
elif gpu_info[0].find('Tesla V100') >= 0:
  max_batch_size = 3
  print('Tesla V100 detected, max_batch_size set to 3')
else:
  max_batch_size = 3
  print('Different GPU detected, max_batch_size set to 3.')
hps.levels = 3
speed_upsampling = False #@param {type: "boolean"}
if speed_upsampling == True:
  hps.hop_fraction = [1,1,.125]
else:
  hps.hop_fraction = [.5,.5,.125]

vqvae, *priors = MODELS[model]
vqvae = make_vqvae(setup_hparams(vqvae, dict(sample_length = 1048576)), device)
top_prior = make_prior(setup_hparams(priors[-1], dict()), vqvae, device)

# The default mode of operation.
# Creates songs based on artist and genre conditioning.
mode = 'primed' #@param ["ancestral", "primed"]
if mode == 'ancestral':
  codes_file=None
  audio_file=None
  prompt_length_in_seconds=None
if mode == 'primed':
  codes_file=None
  # Specify an audio file here.
  audio_file = '/content/gdrive/MyDrive/musicAI/Amen-break.wav' #@param {type:"string"}
  # Specify how many seconds of audio to prime on.
  prompt_length_in_seconds=6 #@param {type:"integer"}

sample_length_in_seconds = 24 #@param {type:"integer"}

if os.path.exists(hps.name):
  # Identify the lowest level generated and continue from there.
  for level in [0, 1, 2]:
    data = f"{hps.name}/level_{level}/data.pth.tar"
    if os.path.isfile(data):
      codes_file = data
      if int(sample_length_in_seconds) > int(librosa.get_duration(filename=f'{hps.name}/level_2/item_0.wav')):
        mode = 'continue'
      else:
        mode = 'upsample'
      break

print('mode is now '+mode)
if mode == 'continue':
  print('Continuing from level 2')
if mode == 'upsample':
  print('Upsampling from level '+str(level))

sample_hps = Hyperparams(dict(mode=mode, codes_file=codes_file, audio_file=audio_file, prompt_length_in_seconds=prompt_length_in_seconds))

if mode == 'upsample':
  sample_length_in_seconds=int(librosa.get_duration(filename=f'{hps.name}/level_{level}/item_0.wav'))
  data = t.load(sample_hps.codes_file, map_location='cpu')
  zs = [z.cpu() for z in data['zs']]
  hps.n_samples = zs[-1].shape[0]

if mode == 'continue':
  data = t.load(sample_hps.codes_file, map_location='cpu')
  zs = [z.cpu() for z in data['zs']]
  hps.n_samples = zs[-1].shape[0]

hps.sample_length = (int(sample_length_in_seconds*hps.sr)//top_prior.raw_to_tokens)*top_prior.raw_to_tokens
assert hps.sample_length >= top_prior.n_ctx*top_prior.raw_to_tokens, f'Please choose a larger sampling rate'

# Note: Metas can contain different prompts per sample.
# By default, all samples use the same prompt.

select_artist = "pink_floyd" #@param {type:"string"}
select_genre = "psychedelic" #@param {type:"string"}
metas = [dict(artist = select_artist,
            genre = select_genre,
            total_length = hps.sample_length,
            offset = 0,
            lyrics = your_lyrics, 
            ),
          ] * hps.n_samples
labels = [None, None, top_prior.labeller.get_batch_labels(metas, 'cuda')]

sampling_temperature = .96 #@param {type:"number"}

if gpu_info[0].find('Tesla T4') >= 0:
  lower_batch_size = 12
  print('Tesla T4 detected, lower_batch_size set to 12')
elif gpu_info[0].find('Tesla K80') >= 0:
  lower_batch_size = 8
  print('Tesla K80 detected, lower_batch_size set to 8')
elif gpu_info[0].find('Tesla P100') >= 0:
  lower_batch_size = 16
  print('Tesla P100 detected, lower_batch_size set to 16')
elif gpu_info[0].find('Tesla V100') >= 0:
  lower_batch_size = 16
  print('Tesla V100 detected, lower_batch_size set to 16')
else:
  lower_batch_size = 8
  print('Different GPU detected, lower_batch_size set to 8.')
lower_level_chunk_size = 32
chunk_size = 16 if model in ('5b', '5b_lyrics') else 32
sampling_kwargs = [dict(temp=.99, fp16=True, max_batch_size=lower_batch_size,
                        chunk_size=lower_level_chunk_size),
                    dict(temp=0.99, fp16=True, max_batch_size=lower_batch_size,
                         chunk_size=lower_level_chunk_size),
                    dict(temp=sampling_temperature, fp16=True, 
                         max_batch_size=max_batch_size, chunk_size=chunk_size)]

if sample_hps.mode == 'ancestral':
  zs = [t.zeros(hps.n_samples,0,dtype=t.long, device='cpu') for _ in range(len(priors))]
  zs = _sample(zs, labels, sampling_kwargs, [None, None, top_prior], [2], hps)
elif sample_hps.mode == 'upsample':
  assert sample_hps.codes_file is not None
  # Load codes.
  data = t.load(sample_hps.codes_file, map_location='cpu')
  zs = [z.cpu() for z in data['zs']]
  assert zs[-1].shape[0] == hps.n_samples, f"Expected bs = {hps.n_samples}, got {zs[-1].shape[0]}"
  del data
  print('One click upsampling!')
elif sample_hps.mode == 'primed':
  assert sample_hps.audio_file is not None
  audio_files = sample_hps.audio_file.split(',')
  duration = (int(sample_hps.prompt_length_in_seconds*hps.sr)//top_prior.raw_to_tokens)*top_prior.raw_to_tokens
  x = load_prompts(audio_files, duration, hps)
  zs = top_prior.encode(x, start_level=0, end_level=len(priors), bs_chunks=x.shape[0])
  zs = _sample(zs, labels, sampling_kwargs, [None, None, top_prior], [2], hps)
elif sample_hps.mode == 'continue':
  data = t.load(sample_hps.codes_file, map_location='cpu')
  zs = [z.cuda() for z in data['zs']]
  zs = _sample(zs, labels, sampling_kwargs, [None, None, top_prior], [2], hps)
else:
  raise ValueError(f'Unknown sample mode {sample_hps.mode}.')

# Set this False if you are on a local machine that has enough memory (this allows you to do the
# lyrics alignment visualization during the upsampling stage). For a hosted runtime, 
# we'll need to go ahead and delete the top_prior if you are using the 5b_lyrics model.
if True:
  del top_prior
  empty_cache()
  top_prior=None
upsamplers = [make_prior(setup_hparams(prior, dict()), vqvae, 'cpu') for prior in priors[:-1]]
labels[:2] = [prior.labeller.get_batch_labels(metas, 'cuda') for prior in upsamplers]

zs = upsample(zs, labels, sampling_kwargs, [*upsamplers, top_prior], hps)

Mounted at /content/gdrive
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/craftmine1000/jukebox-saveopt.git to /tmp/pip-req-build-xbd4n5s7
  Running command git clone -q https://github.com/craftmine1000/jukebox-saveopt.git /tmp/pip-req-build-xbd4n5s7
     |████████████████████████████████| 60 kB 5.5 MB/s 
     |████████████████████████████████| 238 kB 29.6 MB/s 
     |████████████████████████████████| 3.6 MB 70.2 MB/s 
     |████████████████████████████████| 1.6 MB 85.6 MB/s 
     |████████████████████████████████| 2.5 MB 56.8 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 20.2 MB 11.7 MB/s 
     |████████████████████████████████| 3.1 MB 76.4 MB/s 
     |████████████████████████████████| 3.1 MB 94.3 MB/s 
     |████████████████████████████████| 3.1 MB 90.1 MB/s 
  Created wheel for j

# Guide to the above settings:

**your_lyrics:** Specify the lyrics Jukebox should attempt to follow. You can paste any lyrics you want in here or leave it blank, which will result in gibberish.

**model:**
OpenAI has trained a few different models for Jukebox. In this notebook, you can access the 5b_lyrics, 5b and 1b_lyrics models. As you can imagine, the 5b_lyrics model is the superior one, but also requires a stronger GPU to run properly. Which model you should choose depends on the GPU you were assigned, which you can check in the first cell of the notebook. Recommended settings: 5b_lyrics on P100 or T4 GPU, 1b_lyrics on K80 GPU.
(5b_lyrics theoretically works on a K80 now, but sampling is going to be super slow.)

(5b is like 5b_lyrics, without supporting custom lyrics, so it will generate gibberish lyrics)

**hps.n_samples:**
Here you can choose how many samples you want to generate. Different GPUs can handle a different amount of samples. Recommended settings:
P100 GPU: 3 samples,
T4 GPU: 2 samples;
K80 GPU: up to 8 samples, but 1b_lyrics only.

**hps.name:** Specifies the name of the folder in Google Drive, where you will find your results in. Make sure to choose a different name for each of your runs, or else the notebook will get confused.

**speed_upsampling:** If selected, will upsample much faster, at the cost of the samples sounding slightly "choppy". 

**mode:** Available modes are primed and ancestral. Primed will continue an already existing song, ancestral generates a song from scratch. (Upsample mode will be selected automatically if a data file is detected within the folder provided)

**audio_file:** Only needed for primed mode. Specifies which song Jukebox will continue. Upload the file you want (preferred .wav format, but mp3 is supported as well) to the root directory of your Google Drive and fill in its name above.

**prompt_length_in_seconds:** Only needed for primed mode. Specifies how many seconds of your file Jukebox will be primed on (so, at which point Jukebox  will "kick in"). Recommended to keep below 24 seconds for memory reasons.

**sample_length_in_seconds:** Specifies how long your fully generated samples are going to be.

**select_artist and select_genre:** List of available artists and genres can be found here: https://github.com/openai/jukebox/tree/master/jukebox/data/ids
The 5b_lyrics (and 5b) model utilizes the v2 lists, the 1b_lyrics model the v3 lists. It is possible to combine up to five v2 genres, for example "Hip Hop Pop Punk Disco". Combining v3 genres is not possible.

**sampling_temperature:** Determines the creativity and energy of Jukebox. The higher the temperature, the more chaotic and intense the result will be. You can experiment with this. Recommended to keep between .96 and .999

Important links:

Official blog: https://openai.com/blog/jukebox/
Original repo: https://github.com/openai/jukebox/

License: Non-commercial, for details see: https://github.com/openai/jukebox/blob/master/LICENSE